<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/BERTsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive #drive.flush_and_unmount()
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')

Mounted at /content/drive


In [0]:
import zipfile
zip1 = zipfile.ZipFile('/content/drive/My Drive/CSIT Internship/code/raw/DMQA/cnn.zip','r')
zip1.extractall('/content/dmqa')
zip1.close()
zip2 = zipfile.ZipFile('/content/drive/My Drive/CSIT Internship/code/raw/DMQA/dailymail_stories.zip','r')
zip2.extractall('/content/dmqa')
zip2.close()

In [0]:
!pip install --quiet transformers #--force-reinstall

     |████████████████████████████████| 368kB 6.7MB/s 
     |████████████████████████████████| 1.0MB 42.4MB/s 
     |████████████████████████████████| 860kB 47.1MB/s 
     |████████████████████████████████| 645kB 49.0MB/s 


In [0]:
%tensorflow_version 2.x
import os
os.chdir('/content/drive/My Drive/CSIT Internship/code')
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import pandas as pd
from torch import nn
from torch.optim import Adam
from transformers import PreTrainedEncoderDecoder
#from transformers import AutoModel, AutoTokenizer, AutoConfig
from tqdm import tqdm, trange
import contextlib

class DummyFile(object):
  file = None
  def __init__(self, file):
    self.file = file

  def write(self, x):
    # Avoid print() second call (useless \n)
    if len(x.rstrip()) > 0:
        tqdm.write(x, file=self.file)

@contextlib.contextmanager
def nostdout():
    save_stdout = sys.stdout
    sys.stdout = DummyFile(sys.stdout)
    yield
    sys.stdout = save_stdout

TensorFlow 2.x selected.


In [0]:
#from transformers import RobertaConfig,RobertaTokenizer,RobertaModel,RobertaForMaskedLM
#from transformers import DistilBertConfig,DistilBertTokenizer,DistilBertModel,DistilBertForMaskedLM
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, BertModel, PreTrainedEncoderDecoder

In [0]:
torch.cuda.empty_cache()
pretrained = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrained)
#config = DistilBertConfig.from_pretrained(pretrained)
encoder = BertModel.from_pretrained(pretrained)
decoder = BertForMaskedLM.from_pretrained(pretrained)

100%|██████████| 440473133/440473133 [00:14<00:00, 29437135.16B/s]


In [0]:
from transformers import AlbertTokenizer, AlbertModel
torch.cuda.empty_cache()
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
encoder = AlbertModel.from_pretrained('albert-base-v2')
decoder = BertForMaskedLM(BertConfig.from_pretrained('bert-base-uncased'))

There is currently an upstream reproducibility issue with ALBERT v2 models. Please see https://github.com/google-research/google-research/issues/119 for more information.


In [0]:
model = PreTrainedEncoderDecoder.from_pretrained(**{'encoder_model':encoder,'decoder_model':decoder}).to(device)

In [0]:
#https://github.com/huggingface/transformers/blob/35ff345fc9df9e777b27903f11fa213e4052595b/examples/run_summarization_finetuning.py#L139
#modified for Albert, some inference was made & code was modified
class BertSumOptimizer(object):
    """ Specific optimizer for BertSum.
    As described in [1], the authors fine-tune BertSum for abstractive
    summarization using two Adam Optimizers with different warm-up steps and
    learning rate. They also use a custom learning rate scheduler.
    [1] Liu, Yang, and Mirella Lapata. "Text summarization with pretrained encoders."
        arXiv preprint arXiv:1908.08345 (2019).
    """

    def __init__(self, model, lr, warmup_steps, beta_1=0.99, beta_2=0.999, eps=1e-8):
        self.encoder = model.encoder
        self.decoder = model.decoder
        self.lr = lr
        self.warmup_steps = warmup_steps

        self.optimizers = {
            "encoder": Adam(
                model.encoder.parameters(),
                lr=lr["encoder"],
                betas=(beta_1, beta_2),
                eps=eps,
            ),
            "decoder": Adam(
                model.decoder.parameters(),
                lr=lr["decoder"],
                betas=(beta_1, beta_2),
                eps=eps,
            ),
        }

        self._step = 0

    def _update_rate(self, stack):
        return self.lr[stack] * min(
            self._step ** (-0.5), self._step * self.warmup_steps[stack] ** (-0.5)
        )

    def zero_grad(self):
        #There was a bug here. interestingly
        self.optimizers['decoder'].zero_grad()
        self.optimizers['encoder'].zero_grad()

    def step(self):
        self._step += 1
        for stack, optimizer in self.optimizers.items():
            new_rate = self._update_rate(stack)
            for param_group in optimizer.param_groups:
                param_group["lr"] = new_rate
            optimizer.step()

In [0]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location('summary_utils','./utils_summarization.py')
summary_utils = importlib.util.module_from_spec(spec)
sys.modules['summary_utils'] = summary_utils
spec.loader.exec_module(summary_utils)

In [0]:
def collate(data, tokenizer, block_size):
    # remove the files with empty an story/summary, encode and fit to block
    data = filter(lambda x: not (len(x[0]) == 0 or len(x[1]) == 0), data)
    data = [summary_utils.encode_for_summarization(story, summary, tokenizer) for story, summary in data]
    data = [(
            summary_utils.fit_to_block_size(story, block_size, tokenizer.pad_token_id),
            summary_utils.fit_to_block_size(summary, block_size, tokenizer.pad_token_id),
        ) for story, summary in data
    ]

    stories = torch.tensor([story for story, summary in data])
    summaries = torch.tensor([summary for story, summary in data])
    encoder_token_type_ids = summary_utils.compute_token_type_ids(stories, tokenizer.cls_token_id)
    encoder_mask = summary_utils.build_mask(stories, tokenizer.pad_token_id)
    decoder_mask = summary_utils.build_mask(summaries, tokenizer.pad_token_id)
    lm_labels = summary_utils.build_lm_labels(summaries, tokenizer.pad_token_id)

    return (stories,summaries,encoder_token_type_ids,encoder_mask,decoder_mask,lm_labels)

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import functools
import logging
logger = logging.getLogger(__name__)
df = pd.read_pickle('./raw/myNews.pkl')
df = df.sample(frac=1) #apparently this is shuffling
batch_size = 4
epochs = 1
def train(model):
    #straight up stolen still
    lr = {"encoder": 0.002, "decoder": 0.2}
    warmup_steps = {"encoder": 20000, "decoder": 10000}
    optimizer = BertSumOptimizer(model, lr, warmup_steps)
    epoch_iter = trange(epochs,desc='Epoch',disable=True)

    train_dataset = summary_utils.CNNDailyMailDataset(tokenizer, data_dir='/content/dmqa')
    train_sampler = RandomSampler(train_dataset)
    model_collate_fn = functools.partial(collate, tokenizer=tokenizer, block_size=512)
    train_dataloader = DataLoader(train_dataset,sampler=train_sampler,batch_size=batch_size,collate_fn=model_collate_fn)
    t_total = len(train_dataloader)//1*epochs

    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", epochs)
    
    model.train()
    global_step = 0
    tr_loss = 0.0
    for i in epoch_iter:
        train_iter = tqdm(train_dataloader, desc="Epoch %d/%d"%(i,epochs), unit='batches',postfix={'loss':'0.00'},file=sys.stdout,dynamic_ncols=True)
        for step,batch in enumerate(train_iter):
            source, target, encoder_token_type_ids, encoder_mask, decoder_mask, lm_labels = batch
            source = source.to(device)
            target = target.to(device)
            encoder_token_type_ids = encoder_token_type_ids.to(device)
            encoder_mask = encoder_mask.to(device)
            decoder_mask = decoder_mask.to(device)
            lm_labels = lm_labels.to(device)

            model.zero_grad()
            optimizer.zero_grad()

            outputs = model(source,target,
                encoder_token_type_ids=encoder_token_type_ids,
                encoder_attention_mask=encoder_mask,
                decoder_attention_mask=decoder_mask,
                decoder_lm_labels=lm_labels
            )
            loss = outputs[0]
            train_iter.set_postfix(loss='%.2f'%loss.item(),refresh=False)
            loss.backward()
            tr_loss += loss.item()

            torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            optimizer.step()

            global_step += 1
    return global_step,tr_loss/global_step

In [0]:
#https://www.aclweb.org/anthology/D19-1387.pdf
train(model)

In [0]:
#model.eval()
model.save_pretrained("/content/drive/My Drive/CSIT Internship/code/albertBertsum/")

In [0]:
model2 = PreTrainedEncoderDecoder.from_pretrained(
    encoder_pretrained_model_name_or_path = '/content/drive/My Drive/CSIT Internship/code/albertBertsum/encoder/',
    decoder_pretrained_model_name_or_path = '/content/drive/My Drive/CSIT Internship/code/albertBertsum/decoder/')

In [0]:
model2.

In [0]:
options = {
    'max_length':                   512,            #causes overflow
    'stride':                       0,              #how many tokens before to start from in a overflow
    'add_special_tokens':           True,           #Adds [CLS] & [SEP]
    'truncation_strategy':          'only_second',  #Only body of article is cut
    'return_token_type_ids':        True,           #when using pairs, adds the type indicator
    'return_overflowing_tokens':    False,          #allows for overflowing tokens
    'return_special_tokens_mask':   True,           #masking tokens
    'return_tensors':               'pt'            #return as torch tensor
}
pad_c = tokenizer.special_tokens_map['pad_token']
sep_c = tokenizer.special_tokens_map['sep_token']
options['max_length'] += 1

def processInput(articles):
    id_batch = []
    type_batch = []
    mask_batch = []
    for title,text in articles:
        enc = tokenizer.encode_plus(title,text+sep_c+pad_c*options['max_length'],**options)

        token_id = enc['input_ids'][0][:options['max_length']-1]
        token_type = enc['token_type_ids'][0][:options['max_length']-1]
        mask = token_id.clone()
        mask[mask != pad_c] = 1

        id_batch.append(token_id.to(device))
        type_batch.append(token_type.to(device))
        mask_batch.append(mask.to(device))
    return (torch.stack(id_batch),torch.stack(type_batch),torch.stack(mask_batch))
#processInput(df.head(5).apply(lambda row: (row.title,row.text),axis=1))

In [0]:
pretrained = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(pretrained)
encoder = AutoModel.from_pretrained(pretrained)
decoder = AutoModel.from_pretrained(pretrained) #tbh shouldnt this be zero-initialized
encoder = encoder.to(device)
decoder = decoder.to(device)
#encoder.train(), decoder.train()
df = pd.read_pickle('./raw/myNews.pkl')

100%|██████████| 760289/760289 [00:00<00:00, 1829727.12B/s]
There is currently an upstream reproducibility issue with ALBERT v2 models. Please see https://github.com/google-research/google-research/issues/119 for more information.
100%|██████████| 47376696/47376696 [00:02<00:00, 20413045.95B/s]
There is currently an upstream reproducibility issue with ALBERT v2 models. Please see https://github.com/google-research/google-research/issues/119 for more information.


In [0]:
class albertVAE(nn.Module):
    def __init__(self,encoder,decoder,projection_size=8192,vocab_size=30000,device=device):
        super(albertVAE,self).__init__()
        self.encoder = encoder.to(device)
        self.decoder = decoder.to(device)

        self.in_lin = nn.Linear(768*512,int(768*512/64)).to(device)
        self.std_lin = nn.Linear(int(768*512/64),projection_size).to(device)
        self.eps_lin = nn.Linear(int(768*512/64),projection_size).to(device)
        self.proj_lin = nn.Linear(projection_size,int(768*512/64)).to(device)
        self.repa_lin = nn.Linear(int(768*512/64),768*512).to(device)
        self.logit_lin = nn.Linear(768,vocab_size).to(device)

    def encode(self,x):
        h = torch.relu(self.in_lin(x))
        return (self.eps_lin(h), self.std_lin(h))

    def reparameterize(self,mu,logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu+eps*std

    def decode(self, x):
        h1 = torch.relu(self.proj_lin(x))
        h2 = torch.sigmoid(self.repa_lin(h1))
        return self.decoder(inputs_embeds=h2.reshape(-1,512,768))

    def forward(input_ids,type_ids,masks):
        x1 = self.encoder(input_ids=input_ids,token_type_ids=type_ids,attention_mask=masks)
        mu, logvar = self.encode(x1.reshape(x1.shape[0],-1))
        x2 = self.reparameterize(mu, logvar)
        return (self.decoder,mu,logvar)




In [0]:
input_ids,type_ids,masks = processInput(df.head(3).apply(lambda row: (row.title,row.text),axis=1))
model = albertVAE(encoder,decoder).to(device)
print(ouput)

RuntimeError: ignored

In [0]:
tokenizer.vocab_size

30000

In [0]:
torch.cuda.empty_cache()

In [0]:
#TODO: Remove the japanese and chinese in my news dataset like how did it get there
#print(df.text[df.text.str.split().str.len() == max(df.text.str.split().str.len())].item())
lmao = df.text.str.split().str.len()
lol = df.text[(lmao <= 512) & (lmao > 256)] #no 2000 word essays please
sum(lol.str.split().str.len())/len(lol.str.split().str.len())
len(lol)

2328

# Indexing
Dataset is laaarge.
* Sentence embeddings from word embeddings, sparse PCA, clustering, vectors for all documents stored in table, easily calculate distance to query.
* Extract noun phrases, match nouns in query to selected phrases for each document.

I am not sure about the fastest approach, nonetheless, minimize false negatives >>> minimizing false positives. Better last stage algorithm then further minimizes false positives.

* Conclusion: i will use ALBERT finetuned on abstractive summarization directly, using the encoded values as the indexing for the large dataset... theoretically this is already damn good, making it then use the original texts for comparison is overkill.
* damnit im reusing the same ALBERT for the paraphraser too later on lol